### Data Import

La data de la transcripción que se utilizará se compone de un archivo csv por cada pieza musical. También había archivos wav con la interpretación de cada tema.  

También venía un archivo con metadata de cada composición, con información de autor, nombre de la pieza, duración, etc.

La data utilizada fue encontrada en Kaggle.

In [ ]:
import pandas as pd
import numpy as np
import glob
import os

#### Metadata

We'll import the **metadata** first to take a peak.

In [ ]:
df_meta = pd.read_csv('./data/raw/musicnet_metadata.csv')
df_meta.head(10)

,id,composer,composition,movement,ensemble,source,transcriber,catalog_name,seconds
0,1727,Schubert,Piano Quintet in A major,2. Andante,Piano Quintet,European Archive,http://tirolmusic.blogspot.com/,OP114,447
1,1728,Schubert,Piano Quintet in A major,3. Scherzo: Presto,Piano Quintet,European Archive,http://tirolmusic.blogspot.com/,OP114,251
2,1729,Schubert,Piano Quintet in A major,4. Andantino - Allegretto,Piano Quintet,European Archive,http://tirolmusic.blogspot.com/,OP114,444
3,1730,Schubert,Piano Quintet in A major,5. Allegro giusto,Piano Quintet,European Archive,http://tirolmusic.blogspot.com/,OP114,368
4,1733,Schubert,Piano Sonata in A major,2. Andantino,Solo Piano,Museopen,Segundo G. Yogore,D959,546
5,1734,Schubert,Piano Sonata in A major,3. Scherzo. Allegro vivace,Solo Piano,Museopen,Segundo G. Yogore,D959,325
6,1735,Schubert,Piano Sonata in A major,4. Rondo. Allegretto,Solo Piano,Museopen,Segundo G. Yogore,D959,714
7,1739,Schubert,Piano Trio in B-flat major,4. Rondo. Allegro vivace,Piano Trio,European Archive,harfesoft.de,OP99,490
8,1742,Schubert,String Quintet in C major,2. Adagio,Viola Quintet,European Archive,harfesoft.de,OP163,924
9,1749,Schubert,Piano Sonata in A minor,1. Moderato,Solo Piano,Museopen,Segundo G. Yogore,D845,696


In the column `composer`, almost half of the scores are Beethoven's, and Bach has got quite a few also. We'll check this one to see what analysis by composer we can make.  

We have some other interesting fields too. The column `id` links this dataset with each score dataset.   
From `composition` we might get what key is the piece in. Gotta dig in that column later.  
Column `seconds` shows the duration of each piece.
  
I don't think `transcriber` is gonna do much for us. Neither the `catalog_name` nor `source`.

First, we check some basic attributes of our dataset, like dimension and data types. Then we check there are no duplicated rows nor NAs.

In [ ]:
df_meta.shape

(330, 9)

In [ ]:
df_meta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330 entries, 0 to 329
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            330 non-null    int64 
 1   composer      330 non-null    object
 2   composition   330 non-null    object
 3   movement      330 non-null    object
 4   ensemble      330 non-null    object
 5   source        330 non-null    object
 6   transcriber   330 non-null    object
 7   catalog_name  330 non-null    object
 8   seconds       330 non-null    int64 
dtypes: int64(2), object(7)
memory usage: 23.3+ KB


In [ ]:
# Verifying there are no duplicated rows
df_meta.duplicated().value_counts()

False    330
dtype: int64

In [ ]:
# Checking there are no NAs
df_meta.isna().any(axis=1).value_counts()

False    330
dtype: int64

#### Score datasets

Let's check one of the <b>score datasets</b> now to see what we've got.

In [ ]:
# First score dataset is imported
df_1 = pd.read_csv('./data/raw/scores/1727.csv')
df_1

,start_time,end_time,instrument,note,start_beat,end_beat,note_value
0,9182,90078,43,53,4.000,1.500,Dotted Quarter
1,9182,33758,42,65,4.000,0.500,Eighth
2,9182,62430,1,69,4.000,1.000,Quarter
3,9182,202206,44,41,4.000,3.500,Whole
4,9182,62430,1,81,4.000,1.000,Quarter
...,...,...,...,...,...,...,...
6575,19196894,19421150,44,29,365.000,3.000,Dotted Half
6576,19226590,19233758,1,60,365.375,0.125,Thirty Second
6577,19226590,19233758,1,48,365.375,0.125,Thirty Second
6578,19233758,19421150,1,65,365.500,2.500,Dotted Half


In [ ]:
df_1.shape

(6580, 7)

In [ ]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6580 entries, 0 to 6579
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   start_time  6580 non-null   int64  
 1   end_time    6580 non-null   int64  
 2   instrument  6580 non-null   int64  
 3   note        6580 non-null   int64  
 4   start_beat  6580 non-null   float64
 5   end_beat    6580 non-null   float64
 6   note_value  6580 non-null   object 
dtypes: float64(2), int64(4), object(1)
memory usage: 360.0+ KB


Interesting. What most catches my attention is the `instrument` and the `note` column. Didn't find a dictionary to decode any of them, but listening to some of the wav files we can get the notes codification, and with the ensemble column in the metadata we might mostly figure out the instruments, too.

I listened to a couple of wav files and couldn't understand the `start_time` and `end_time` columns. They are clearly not in seconds nor miliseconds. Shame.

Another quick listen to some of the audio files led me to the conclusion that the note C is mapped as 0. Thus, C-sharp is 1, D is 2, and so on. 12 would be C again.
  
If we can get the key of each composition from the metadata table, we can get what scales and modes are used.

### Data wrangling: Metadata

#### Creating column "key" 

We'll start extracting from the metadata table (`composition` and `movement` columns) the key of each composition

In [ ]:
# Splitting the column composition
df_meta['composition'].str.split(' in ', expand=True)

,0,1
0,Piano Quintet,A major
1,Piano Quintet,A major
2,Piano Quintet,A major
3,Piano Quintet,A major
4,Piano Sonata,A major
...,...,...
325,Piano Sonata No 10,G major
326,Piano Sonata No 10,G major
327,Violin Partita No 1,B minor
328,Piano Sonata No 9,E major


In [ ]:
# There are some missing values, let's check that
sr_key1 = df_meta['composition'].str.split(' in ', expand=True)[1]
df_meta[sr_key1.isna()]

,id,composer,composition,movement,ensemble,source,transcriber,catalog_name,seconds
19,1763,Schubert,4 Impromptus,1. Impromptu in F minor,Solo Piano,Charlie Albright,Jeruen Espino Dery,OP142,600
20,1764,Schubert,4 Impromptus,2. Impromptu in A-flat major,Solo Piano,Charlie Albright,Jeruen Espino Dery,OP142,438
21,1765,Schubert,4 Impromptus,3. Impromptu in B-flat major,Solo Piano,Charlie Albright,Jeruen Espino Dery,OP142,676
22,1766,Schubert,4 Impromptus,4. Impromptu in F minor,Solo Piano,Charlie Albright,Jeruen Espino Dery,OP142,427
126,2217,Bach,Cello Suite 3,1. Prelude,Solo Cello,European Archive,David J. Grossman,BWV1009,186
127,2218,Bach,Cello Suite 3,2. Allemande,Solo Cello,European Archive,David J. Grossman,BWV1009,199
128,2219,Bach,Cello Suite 3,3. Courante,Solo Cello,European Archive,David J. Grossman,BWV1009,260
129,2220,Bach,Cello Suite 3,4. Sarabande,Solo Cello,European Archive,David J. Grossman,BWV1009,302
130,2221,Bach,Cello Suite 3,5. Bouree,Solo Cello,European Archive,David J. Grossman,BWV1009,241
131,2222,Bach,Cello Suite 3,6. Gigue,Solo Cello,European Archive,David J. Grossman,BWV1009,202


In [ ]:
# We can get the key for the Schubert pieces from the column 'movement'.
# No luck with Bach's (at least, within this dataset)
# We repeat the process and check the rows where the split method didn't get a NA

sr_key2 = df_meta['movement'].str.split(' in ', expand = True)[1]
sr_key2[sr_key2.notna()]

19         F minor
20    A-flat major
21    B-flat major
22         F minor
Name: 1, dtype: object

In [ ]:
# We create the column 'key' from both series.

keys = {'key1': sr_key1, 'key2': sr_key2}
df_keys = pd.DataFrame(keys)        # Converting to frame to operate

df_keys.fillna('', inplace=True)    # Filling null values with empty strings

keys = df_keys['key1'] + df_keys['key2']      # Combining into one series object

keys.value_counts().sort_index()     # Frequency of keys

                          12
A major                   30
A minor                   17
A-flat major               9
B major                    2
B minor                    7
B-flat Major               3
B-flat major              21
B-flat minor               1
C major                   16
C minor                   21
C-sharp major              2
C-sharp minor              7
D Major                    2
D Minor                    3
D major                   15
D minor                   10
D-flat major               3
E major                   12
E minor                   10
E-flat major              35
E-flat major for Winds     7
E-flat minor               1
F                          4
F Major                    4
F major                   15
F minor                   15
F-sharp major              2
F-sharp minor              2
G major                   24
G minor                   16
G-sharp minor              2
dtype: int64

There are 12 blank values for key. They must surely be the Bach compositions that didn't have the key in the columns `composition` nor `movement`.

There are also 4 compositions with key in just F, not major nor minor.
And about the majors, there are some rows with capital M in Major, so we'll standarize that.

There are some rows that indicate key in "E-flat major for Winds". That "for Winds" at the end kinda rattles me, so it will be stripped.

In [ ]:
# Checking the rows with an empty string as key
df_meta[keys == '']

,id,composer,composition,movement,ensemble,source,transcriber,catalog_name,seconds
126,2217,Bach,Cello Suite 3,1. Prelude,Solo Cello,European Archive,David J. Grossman,BWV1009,186
127,2218,Bach,Cello Suite 3,2. Allemande,Solo Cello,European Archive,David J. Grossman,BWV1009,199
128,2219,Bach,Cello Suite 3,3. Courante,Solo Cello,European Archive,David J. Grossman,BWV1009,260
129,2220,Bach,Cello Suite 3,4. Sarabande,Solo Cello,European Archive,David J. Grossman,BWV1009,302
130,2221,Bach,Cello Suite 3,5. Bouree,Solo Cello,European Archive,David J. Grossman,BWV1009,241
131,2222,Bach,Cello Suite 3,6. Gigue,Solo Cello,European Archive,David J. Grossman,BWV1009,202
158,2293,Bach,Cello Suite 4,1. Prelude,Solo Cello,European Archive,David J. Grossman,BWV1010,310
159,2294,Bach,Cello Suite 4,2. Allemande,Solo Cello,European Archive,David J. Grossman,BWV1010,228
160,2295,Bach,Cello Suite 4,3. Courante,Solo Cello,European Archive,David J. Grossman,BWV1010,259
161,2296,Bach,Cello Suite 4,4. Sarabande,Solo Cello,European Archive,David J. Grossman,BWV1010,291


In [ ]:
# Checking the rows with key in just F
df_meta[keys == 'F']

,id,composer,composition,movement,ensemble,source,transcriber,catalog_name,seconds
102,2177,Ravel,String Quartet in F,1. Allegro moderato - tres doux,String Quartet,Musicians from Marlboro,Gregory Richardson,35,490
103,2178,Ravel,String Quartet in F,2. Assez vif - tres rythme,String Quartet,Musicians from Marlboro,Gregory Richardson,35,355
104,2179,Ravel,String Quartet in F,3. Tres lent,String Quartet,Musicians from Marlboro,Gregory Richardson,35,505
105,2180,Ravel,String Quartet in F,4. Vif et agite,String Quartet,Musicians from Marlboro,Gregory Richardson,35,293


Ok, these Ravel compositions are in just F.

In [ ]:
# Removing the ' for Winds' suffix and standarizing capitalization
keys = keys.str.replace(' for Winds','').str.replace('M', 'm')
keys.value_counts().sort_index()

                 12
A major          30
A minor          17
A-flat major      9
B major           2
B minor           7
B-flat major     24
B-flat minor      1
C major          16
C minor          21
C-sharp major     2
C-sharp minor     7
D major          17
D minor          13
D-flat major      3
E major          12
E minor          10
E-flat major     42
E-flat minor      1
F                 4
F major          19
F minor          15
F-sharp major     2
F-sharp minor     2
G major          24
G minor          16
G-sharp minor     2
dtype: int64

And thus, we're good to go with our column `key`

#### Tonic columns

In order to analyze the notes and scales used, we need a column for the tonic so we can establish the relationship between each of the notes and the tonic.  

We can get the tonic from the keys column we just made (technically is a series by now, but it will be a column in a while). We need just the tonic, so we'll take out the majors and minors. Then, we will create a column with the tonic encoded to an integer, according to the system used in the notes column in the score datasets.

In [ ]:
tonic = keys.str.replace(' major','').str.replace(' minor','')    # Creating the tonic series

# Now, we need to create the encoded tonic series.
# For this, we will make a dictionary.

sorted(tonic.unique())

['',
 'A',
 'A-flat',
 'B',
 'B-flat',
 'C',
 'C-sharp',
 'D',
 'D-flat',
 'E',
 'E-flat',
 'F',
 'F-sharp',
 'G',
 'G-sharp']

In [ ]:
# Dictionary to map the notes to the system used
dict_notes = {
    'C': 0,
    'C-sharp': 1,
    'D': 2,
    'D-flat': 1,
    'E': 4,
    'E-flat': 3,
    'F': 5,
    'F-sharp': 6,
    'G': 7,
    'G-sharp': 8,
    'A': 9,
    'A-flat': 8,
    'B': 11,
    'B-flat': 10}

tonic_encoded = tonic.replace(to_replace = dict_notes)

tonic_encoded

0       9
1       9
2       9
3       9
4       9
       ..
325     7
326     7
327    11
328     4
329     7
Length: 330, dtype: object

We set now the metadata dataframe with the 3 columns created, and drop the columns with no use.

In [ ]:
# Generating a frame with just the new columns
df_key = pd.DataFrame({
    'key': keys,
    'tonic': tonic,
    'tonic_encoded': tonic_encoded})

# Concatenating the new columns to the metadata frame. Then, dropping the columns that serve no purpose.
df = pd.concat([df_meta, df_key], axis = 1).drop(columns=['source', 'transcriber', 'catalog_name'])

In [ ]:
# Ok, now for the blanks in the column `key`, from those Bach compositions. We get rid of them.
df=df[df.key!='']

In [ ]:
# Saving the dataframe into a csv
df.to_csv('./data/processed/scores_metadata.csv', index=False)

df

,id,composer,composition,movement,ensemble,seconds,key,tonic,tonic_encoded
0,1727,Schubert,Piano Quintet in A major,2. Andante,Piano Quintet,447,A major,A,9
1,1728,Schubert,Piano Quintet in A major,3. Scherzo: Presto,Piano Quintet,251,A major,A,9
2,1729,Schubert,Piano Quintet in A major,4. Andantino - Allegretto,Piano Quintet,444,A major,A,9
3,1730,Schubert,Piano Quintet in A major,5. Allegro giusto,Piano Quintet,368,A major,A,9
4,1733,Schubert,Piano Sonata in A major,2. Andantino,Solo Piano,546,A major,A,9
...,...,...,...,...,...,...,...,...,...
325,2632,Beethoven,Piano Sonata No 10 in G major,2. Andante,Solo Piano,341,G major,G,7
326,2633,Beethoven,Piano Sonata No 10 in G major,3. Scherzo: Allegro assai,Solo Piano,227,G major,G,7
327,2659,Bach,Violin Partita No 1 in B minor,6. Double,Solo Violin,108,B minor,B,11
328,2677,Beethoven,Piano Sonata No 9 in E major,1. Allegro,Solo Piano,445,E major,E,4


### Data Wrangling: Score Datasets
There's a csv file with the score for each composition. In this section, we just have to concatenate all the score datasets into one dataframe, creating a column with the `id` of the composition. The id can be found in the name of each csv file.

This is the best way I came up with to do this. Doesn't feel too efficient, so I'll probably get back to this topic.

In [ ]:
# Defining the path for every csv file and storing them in a list
scores_path = './data/raw/scores'
scores_list = glob.glob(os.path.join(scores_path, '*.csv'))

# Creating dictionary with id of composition (from filename) as key and it's corresponding dataframe as the value
df_dict = {
    os.path.basename(f)[:-4] : pd.read_csv(f) for f in scores_list
}

# Creating a column with the id for each dataframe in the dictionary
for key in df_dict:
    df_dict[key]['id'] = int(key)

# Concatenating all scores into one dataframe
df_scores = pd.concat(df_dict.values())

# Changing the order of columns to have the id as first column
df_columns = df_scores.columns.tolist()
df_columns = df_columns[-1:] + df_columns[:-1] 
df_scores = df_scores[df_columns]

df_scores

,id,start_time,end_time,instrument,note,start_beat,end_beat,note_value
0,2416,10206,19934,61,63,2.00000,0.6875,Dotted Eighth
1,2416,10206,19934,61,55,2.00000,0.6875,Dotted Eighth
2,2416,24030,31710,61,63,3.00000,0.6875,Dotted Eighth
3,2416,24030,31710,61,55,3.00000,0.6875,Dotted Eighth
4,2416,34782,42462,61,55,4.00000,0.6875,Dotted Eighth
...,...,...,...,...,...,...,...,...
1304,2209,6551006,6572510,1,56,227.13750,0.5000,Quarter
1305,2209,6570973,6597598,1,63,227.61875,0.5000,Quarter
1306,2209,6572510,6598622,1,54,227.63750,0.5000,Quarter
1307,2209,6597598,6864350,1,65,228.11875,2.0000,Whole


In [ ]:
# It worked, so I'll just export it
df_scores.to_csv('./data/processed/scores.csv', index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=177ea176-1f94-4265-9666-0cca06278d8e' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>